<a href="https://colab.research.google.com/github/nobububi/Brandear/blob/master/code/03_02_%E5%9F%BA%E6%9C%AC%E7%89%B9%E5%BE%B4%E9%87%8F%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import math

import pandas as pd
import numpy as np
from google.colab import drive

In [2]:
DATA_DIR = "drive/MyDrive/data/"

INPUT_FILE = f"{DATA_DIR}tiho_work/preprocess/03_01_race_jisseki_df.pickle"
OUTPUT_FILE = f"{DATA_DIR}tiho_work/preprocess/03_02_race_jisseki_df.pickle"


In [3]:
# 自分のドライブ配下をマウント
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
sys.path.append(f'drive/MyDrive/colab_notebooks/tiho/modules')
import race_id_operator
import change_type

In [5]:
race_jisseki_df = pd.read_pickle(INPUT_FILE)

In [6]:
race_jisseki_df.head()

,データ区分,データ作成年月日,競走コード,馬番,枠番,血統登録番号,馬名,開催年月日,馬記号コード,性別コード,...,障不・着回数,芝16下・着回数,芝22下・着回数,芝22超・着回数,ダ16下・着回数,ダ22下・着回数,ダ22超・着回数,脚質傾向,登録レース数,baoz_ra_kyousoubamasuta_df
0,7,2012-12-18,20501014301,1,1,1999109207,アケボノスター,2005-01-01,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,7,2012-12-18,20501014301,2,2,2000109171,カリスマサンガール,2005-01-01,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,7,2012-12-18,20501014301,3,3,2000109413,アルファモナカアン,2005-01-01,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,7,2012-12-18,20501014301,4,4,2000104939,ゴールドダイナ,2005-01-01,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,7,2012-12-18,20501014301,5,5,2000101705,フェアウェルガッド,2005-01-01,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### なんかレース数少ない？

In [7]:
race_jisseki_df.shape

(3241593, 308)

In [8]:
tmp_df = race_jisseki_df[race_jisseki_df["開催年月日"] == "2018-07-03"].copy()

In [9]:
# 2018/07/03で確認
# https://nar.netkeiba.com/top/race_list.html?kaisai_date=20180703&kaisai_id=2018450703
# パッと見全レースありそう
tmp_df[["競走コード", "馬番"]].drop_duplicates()["競走コード"].value_counts().sort_index()

21807034501    12
21807034502    12
21807034503    12
21807034504    10
21807034505     8
21807034506    12
21807034507    11
21807034508    11
21807034509    14
21807034510    14
21807034511     8
21807034512    12
21807034601    11
21807034602     7
21807034603     9
21807034604     9
21807034605     8
21807034606    10
21807034607    10
21807034608     9
21807034609    10
21807034610     9
21807034611     7
21807034612    10
Name: 競走コード, dtype: int64

### date_int

In [10]:
race_jisseki_df["date_int"] = pd.to_datetime(race_jisseki_df["開催年月日"]).dt.strftime('%Y%m%d').astype(int)

### race_id

In [11]:
print(datetime.datetime.now())
# 1min
race_jisseki_df["RACE_ID"] = (
    race_jisseki_df[["date_int", "競馬場コード", "開催回", "開催日目", "レース番号"]]
    .apply(lambda s: race_id_operator.make_RACEID(s["date_int"], s["競馬場コード"], s["開催回"], s["開催日目"], s["レース番号"]), axis=1)
)
print(datetime.datetime.now())

2022-07-16 06:11:45.571863
2022-07-16 06:13:07.573983


In [12]:
race_jisseki_df[["date_int", "競馬場コード", "開催回", "開催日目", "レース番号", "RACE_ID"]].head()

,date_int,競馬場コード,開催回,開催日目,レース番号,RACE_ID
0,20050101,43,11,1,1,20050101_43_11_1_1
1,20050101,43,11,1,1,20050101_43_11_1_1
2,20050101,43,11,1,1,20050101_43_11_1_1
3,20050101,43,11,1,1,20050101_43_11_1_1
4,20050101,43,11,1,1,20050101_43_11_1_1


### 月日

In [13]:
race_jisseki_df['月'] = race_jisseki_df['date_int'].map(lambda x : int(str(x)[4:6]))
race_jisseki_df['日'] = race_jisseki_df['date_int'].map(lambda x : int(str(x)[6:8]))

In [14]:
race_jisseki_df['月'].value_counts()

5     300570
10    288122
7     280932
6     277213
8     273145
11    271845
4     269364
12    266439
1     263131
9     262847
3     257475
2     230510
Name: 月, dtype: int64

In [15]:
race_jisseki_df['日'].value_counts()

3     111316
23    111241
5     109952
20    109673
24    109647
14    109562
4     109355
21    109042
17    109026
11    108224
15    107922
16    107888
12    106951
18    106562
10    106253
13    106020
22    105994
19    105940
2     105516
28    105002
9     104539
25    104082
26    103550
27    103428
1     102338
8     102081
6     101612
29     99869
7      99808
30     96192
31     63008
Name: 日, dtype: int64

### 馬年齢系

In [16]:
[c for c in race_jisseki_df.columns if "日" in c]

['データ作成年月日',
 '開催年月日',
 '年月日',
 '開催月日',
 '開催日目',
 '曜日コード',
 '競走馬登録年月日',
 '競走馬抹消年月日',
 '生年月日',
 '日']

In [17]:
race_jisseki_df["生年月日"].head()

0    1999-04-28
1    2000-04-19
2    2000-02-24
3    2000-04-04
4    2000-04-23
Name: 生年月日, dtype: object

In [18]:
s = "2000-04-23"
s[0:4] + s[5:7] + s[8:10]

'20000423'

In [19]:
race_jisseki_df['生まれ月'] = race_jisseki_df['生年月日'].map(lambda x : int(str(x)[5:7]) if not pd.isnull(x) else np.nan)
race_jisseki_df['生まれ月_sin'] = race_jisseki_df['生まれ月']\
.map(lambda x: math.sin(math.radians(360 * (x/12))) if not pd.isnull(x) else np.nan)
race_jisseki_df['生まれ月_cos'] = race_jisseki_df['生まれ月']\
.map(lambda x: math.cos(math.radians(360 * (x/12)))if not pd.isnull(x) else np.nan)

race_jisseki_df['馬日齢'] = (
    race_jisseki_df['生年月日'].map(lambda x : 1 if not pd.isnull(x) else np.nan)
    *(
        race_jisseki_df['date_int'].map(lambda x : change_type.change_string_to_date(str(int(x))))
        - race_jisseki_df['生年月日'].map(lambda x : change_type.change_string_to_date(x[0:4] + x[5:7] + x[8:10]) if not pd.isnull(x) else np.nan)
     ).map(lambda x :x.days )
)

In [20]:
race_jisseki_df[["生年月日", '生まれ月', "生まれ月_sin", "生まれ月_cos", "date_int", "馬日齢"]].head()

,生年月日,生まれ月,生まれ月_sin,生まれ月_cos,date_int,馬日齢
0,1999-04-28,4.0,0.866025,-0.5,20050101,2075.0
1,2000-04-19,4.0,0.866025,-0.5,20050101,1718.0
2,2000-02-24,2.0,0.866025,0.5,20050101,1773.0
3,2000-04-04,4.0,0.866025,-0.5,20050101,1733.0
4,2000-04-23,4.0,0.866025,-0.5,20050101,1714.0


In [21]:
race_jisseki_df[['生まれ月', "生まれ月_sin", "生まれ月_cos"]].drop_duplicates()

,生まれ月,生まれ月_sin,生まれ月_cos
0,4.0,8.660254e-01,-5.000000e-01
2,2.0,8.660254e-01,5.000000e-01
5,3.0,1.000000e+00,6.123234e-17
6,5.0,5.000000e-01,-8.660254e-01
7,6.0,1.224647e-16,-1.000000e+00
142,1.0,5.000000e-01,8.660254e-01
489,8.0,-8.660254e-01,-5.000000e-01
3661,10.0,-8.660254e-01,5.000000e-01
4230,7.0,-5.000000e-01,-8.660254e-01
8546,9.0,-1.000000e+00,-1.836970e-16


### 経過日数

In [22]:
print(datetime.datetime.now())
race_jisseki_df = race_jisseki_df.sort_values(['馬名', 'date_int'])
race_jisseki_df['date'] = race_jisseki_df['date_int'].map(lambda x : change_type.change_string_to_date(str(x)))
race_jisseki_df['horse_past_days1'] = (
    (
        race_jisseki_df['date']
        - race_jisseki_df.groupby('馬名')['date'].transform(lambda x : x.shift(1))
    ).map(lambda x :x.days )
)
race_jisseki_df['horse_past_days2'] = (
    (
        race_jisseki_df.groupby('馬名')['date'].transform(lambda x : (x - x.shift(2)).map(lambda x :x.days ))
    )
)
race_jisseki_df['horse_past_days3'] = (
    (
        race_jisseki_df.groupby('馬名')['date'].transform(lambda x : (x - x.shift(3)).map(lambda x :x.days ))
    )
)
race_jisseki_df['horse_past_days4'] = (
    (
        race_jisseki_df.groupby('馬名')['date'].transform(lambda x : (x - x.shift(4)).map(lambda x :x.days ))
    )
)
race_jisseki_df['horse_past_days5'] = (
    (
        race_jisseki_df.groupby('馬名')['date'].transform(lambda x : (x - x.shift(5)).map(lambda x :x.days ))
    )
)
print(datetime.datetime.now())

2022-07-16 06:15:18.686914
2022-07-16 06:25:29.290288


In [23]:
race_jisseki_df[["馬名", "date_int", "horse_past_days1", "horse_past_days2", "horse_past_days3", "horse_past_days4", "horse_past_days5"]].iloc[5000:5100]

,馬名,date_int,horse_past_days1,horse_past_days2,horse_past_days3,horse_past_days4,horse_past_days5
2616252,アイアス,20190109,16.0,41.0,55.0,99.0,127.0
2683131,アイアス,20190527,138.0,154.0,179.0,193.0,237.0
2689941,アイアス,20190610,14.0,152.0,168.0,193.0,207.0
2698853,アイアス,20190628,18.0,32.0,170.0,186.0,211.0
2709342,アイアス,20190717,19.0,37.0,51.0,189.0,205.0
...,...,...,...,...,...,...,...
576871,アイアムアスリーダ,20071118,7.0,20.0,70.0,127.0,182.0
580964,アイアムアスリーダ,20071125,7.0,14.0,27.0,77.0,134.0
588396,アイアムアスリーダ,20071210,15.0,22.0,29.0,42.0,92.0
594403,アイアムアスリーダ,20071222,12.0,27.0,34.0,41.0,54.0


### 成績系

In [24]:
[c for c in race_jisseki_df.columns if "着順" in c]

['確定着順', '前走着順']

In [25]:
race_jisseki_df["確定着順"].value_counts().sort_index()

0      36058
1     299269
2     298657
3     298529
4     298142
5     298058
6     296147
7     290693
8     275127
9     241020
10    191400
11    129336
12     99897
13     62700
14     51894
15     36736
16     27294
17      5968
18      4613
19        18
20        13
21         8
22         8
23         6
24         2
Name: 確定着順, dtype: int64

In [26]:
race_jisseki_df['異常'] = race_jisseki_df['確定着順'].map(lambda x : 1 if x == 0 else 0)
# race_jisseki_df['着順END'] = race_jisseki_df['確定着順'].map(lambda x : x if x !=0 else 8)

### その他

In [27]:
race_jisseki_df['馬番偶数'] =  race_jisseki_df['馬番'] %2 

## 保存

In [28]:
race_jisseki_df.to_pickle(OUTPUT_FILE)